## Planning Graph

In [1]:
import psycopg2
from geojson import LineString, GeometryCollection, Point
from math import radians, sin, cos, sqrt, asin
import heapq
import numpy as np
from osmread import parse_file, Way, Node
import os.path
import pickle

class GraphPlan(object):
    """ Process data for a country to produce data structures needed for
        a graph search
    """
    # TODO: Class excepts any bz2 file for processing 
    def __init__(self):
        """
        Connect to OSM data base and prepare basic data structures
        """
        #TODO: Allow user to specify database, user and national region
        conn_string = "host='localhost' dbname='osm' user='dcromp'"
        print ("Connecting to database\n->%s" % (conn_string))
        # get a connection, if a connect cannot be made an exception will be raised here
        conn = psycopg2.connect(conn_string)
        # conn.cursor will return a cursor object
        self.cursor = conn.cursor()
        print ("Connected!\n")
        
        # Load a file of previously calculated node distances if it exists
        if os.path.isfile("saved_distances.p"):
            self.saved_distances = pickle.load(open("saved_distances.p","rb"))
        else:
            self.saved_distances = {}
            
        
        # User guide on using osmosis tags
        # http://skipperkongen.dk/2012/08/02/examples-of-querying-a-osm-postgresql-table-with-the-hstore-tags-column/
        
    def node_neighbours(self, node_id):
        """
        For a given node_id its neighbouring nodes ids are returned 
        Args:
            Node_id string
        Returns:
            [node_id] List of neighbouring node id's  
        """
        #TODO: David Crompton, Combine the select statements into a signle database query 
        #Query to get way which the node occurs in
        query_string = "SELECT * FROM way_nodes WHERE node_id = {}".format(node_id)
        self.cursor.execute(query_string)
        records = self.cursor.fetchall()
        #Extract way id's from the records
        ways = [record[0] for record in records]
        neighbours_list = set()
        for way in ways:
            #Make sure we return a road using highway key and not a polygon of a building
            query_string = "SELECT nodes FROM ways WHERE id = {} AND exist(tags, 'highway')".format(way)
            self.cursor.execute(query_string)
            records = self.cursor.fetchall()
            try: #Possiable that way is not a road so nodes is empty
                nodes = records[0][0]
            except:
                continue #Skip way if way is not a road
            node_index = nodes.index(int(node_id))
            #Extract the nodes neighbour and save to dict
            forward_node = node_index + 1
            backward_node = node_index - 1
            try:
                forward = nodes[forward_node]
                neighbours_list.add(str(forward))
            except:
                pass
            if backward_node >= 0:
                try:
                    backward = nodes[backward_node]
                    neighbours_list.add(str(backward))
                except:
                    pass 
        return list(neighbours_list)
    
    def distance_measure(self, node_1, node_2):
        """
        Calculates the distance between two nodes
        Args:
            node_1: Node id of the first node
            node_2: Node id of the second node
        Returns:
            float: distance in meters
        """
        if node_1 == node_2: # Nodes are the same
            return 0
        
        # Quick check to see if we have seen these nodes before
        try:
            distance = self.saved_distances[(node_1, node_2)]
            return distance
        except:
            pass
    
        query_string = """
        SELECT ST_AsText(ST_Transform(geom,4326))
        FROM nodes
        WHERE id IN ({}, {})
        """.format(node_1, node_2)
        self.cursor.execute(query_string)
        results = self.cursor.fetchall()
        geom_1 = results[0][0]
        geom_2 = results[1][0]
        query_string = """
        SELECT ST_Distance_Sphere(ST_GeomFromText('{}',4326), ST_GeomFromText('{}',4326))
        """.format(geom_1, geom_2)
        self.cursor.execute(query_string)
        results = self.cursor.fetchall()
        return results[0][0]
    
    def nearest_roads(self, lon, lat, rows=10):
        """
        User can insert any lon lat and return any number of node_id's within the area.
        Args:
            lon - longitude of location
            lat - latitide of location
            rows: Number of random locations within 1Km to be returned
        Returns:
            [node_id,...] -  The nearest node_id's
        """
        # 0.01 units is rought 1.11 Km
        query_string = """
        SELECT nodes.id
        FROM nodes, way_nodes, ways
        WHERE ST_DWithin(nodes.geom, ST_GeomFromText('POINT({} {})',4326), 0.01) 
        AND nodes.id = way_nodes.node_id 
        AND way_nodes.way_id = ways.id 
        AND exist(ways.tags, 'highway')
        ORDER BY random()
        LIMIT {}; 
        """.format(lon, lat, rows)
        self.cursor.execute(query_string)
        records = self.cursor.fetchall()
        print(records)
        
    def nearest_road(self, lon, lat):
        """
        User can insert any lon lat and the nearest node_id that is on a road is returned
        Args:
            lon - longitude of location
            lat - latitide of location
        Returns:
            node_id -  The nearest node to lon/lat input
        """
        #TODO David Crompton: Fix returns empty if no node within 0.01 degress (1.11Km)
        query_string = """
        SELECT nodes.id
        FROM nodes, way_nodes, ways
        WHERE ST_DWithin(nodes.geom, ST_GeomFromText('POINT({} {})',4326), 0.01) 
        AND nodes.id = way_nodes.node_id 
        AND way_nodes.way_id = ways.id 
        AND exist(ways.tags, 'highway')
        ORDER BY ST_Distance(nodes.geom, ST_GeomFromText('POINT({} {})',4326)) ASC
        LIMIT 1;
        """.format(lon, lat, lon, lat)
        
        print("Warning very slow....")
        self.cursor.execute(query_string)
        records = self.cursor.fetchall()
        print("Found nearest node id :)")
        return records
    
    def node_path_geojson(self, node_list):
        """
        Takes a list of node id's and converts them into a goejson linestring that can be displayed on maps
        Args:
            [node_id,...] : list of node id's
        Returns:
            LineString([lat, lng,...]) : A linestring of geographic coordinates in geojson format
        """
        roads = []
        for node_id in node_list:
            node_id = node_id.split('_BUS_')[0]
            query_string = "SELECT ST_AsText(ST_Transform(geom,4326)) FROM nodes WHERE id = {}".format(node_id)
            self.cursor.execute(query_string)
            records = self.cursor.fetchall()
            lng, lat = records[0][0].replace('POINT(', ' ').replace(')', '').split(' ')[1:3]
            roads.append((float(lng), float(lat)))
        return LineString(roads)
    
    def save_distances(self):
        pickle.dump(self.saved_distances, open("saved_distances.p", "wb" ))
    
class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]
        
graph = GraphPlan()

Connecting to database
->host='localhost' dbname='osm' user='dcromp'
Connected!



In [2]:
graph.nearest_road(10.7522, 59.9139)

Warning very slow....
Found nearest node id :)


[(1241388716,)]

In [ ]:
SELECT nodes.id
FROM nodes, way_nodes, ways
WHERE ST_DWithin(nodes.geom, ST_GeomFromText('POINT(10.7522 59.9139)',4326), 0.01) 
AND nodes.id = way_nodes.node_id 
AND way_nodes.way_id = ways.id 
AND exist(ways.tags, 'highway')
ORDER BY random()
LIMIT 10;

## Bus Graph

In [ ]:
class BusPlan(GraphPlan):
    """ 
    Extract a BusPlan from the core GraphPlan
    """
    def __init__(self):
        GraphPlan.__init__(self)
        
    def bus_nodes(self, node_id):
        """
        Takes a node_id and returns the bus routes if any the node_id belongs to.
        Args:
            Node_id
        Returns:
            [bus_route_id,..]: List of new node_id's with the bus route appended
            None if node is not part of a bus route
        """
        bus_routes = []
        # Select all the bus routes the node appears in
        query_string = """
        SELECT DISTINCT relations.tags->'name', relations.tags->'operator', relations.tags->'ref'
        FROM way_nodes, relation_members, relations 
        WHERE {} = way_nodes.node_id
        AND way_nodes.way_id = relation_members.member_id
        AND relation_members.relation_id = relations.id
        AND tags @> '"route"=>"bus"'::hstore""".format(node_id)
        self.cursor.execute(query_string)
        results = self.cursor.fetchall()
        if len(results) > 0: #Node is part of a bus route 
            for bus_route in results:
                bus_routes.append(str(node_id) + '_BUS_{}_{}'.format(bus_route[1], bus_route[2]))
        return bus_routes
    
    def bus_stop(self, node_id):
        query_string = """
        SELECT * 
        FROM nodes 
        WHERE id = {} 
        AND tags @> '"public_transport"=>"stop_position"'::hstore;""".format(node_id)
        self.cursor.execute(query_string)
        results = self.cursor.fetchall()
        if len(results) > 0: # Node is a bus stop
            return True
        else:
            return False # Node is not a bus stop
    
    
    def bus_neighbours(self, node_id):
        """
        Extends the node_neighbours function to also return the children of a bus search graph. In short it seperates
        out bus routes from the rest of the graph
        Args:
            node_id: A float representing the node
        Returns:
            [node_id]: A list of neighbouring node_id's and bus routes in the format node_id_BUS_operator_ref
        """
        neighbours = []
        bus = False
        #Is node_id a normal node or a bus node
        try: #Node is a bus
            bus_id = node_id.split('_BUS_')[1]
            bus = True
        except: # Regular node
            pass
        if bus == True:
            raw_node_id = node_id.split('_BUS_')[0]
            stop_test = self.bus_stop(raw_node_id) # Test if node is a bus stop
            if stop_test == True:
                neighbours.append(raw_node_id)
            children = self.node_neighbours(int(raw_node_id))
            for child in children:
                bus_neighbours = self.bus_nodes(child)
                # Only bus nodes on the same route are valid neighbours
                bus_neighbours = [node for node in bus_neighbours if node.split('_BUS_')[1] == bus_id]
                neighbours.extend(bus_neighbours)
            return neighbours 
        else:
            raw_node_id = node_id.split('_BUS_')[0]
            children = self.node_neighbours(int(raw_node_id))
            neighbours.extend(children)
            
            stop_test = self.bus_stop(raw_node_id) # Test if node is a bus stop
            if stop_test == False:
                return neighbours
            else:
                neighbours.extend(self.bus_nodes(raw_node_id))
                return neighbours
    
    def bus_distance(self, node_1, node_2):
        """
        Allows the use of the distance_measure function with bus route node_id's
        Args:
            node_1: A node_id or a node_id with an appended bus route
            node_2: A node_id or a node_id with an appended bus route
        Returns:
            meters: distance between the two nodes
        """
        raw_node_1 = node_1.split('_BUS_')[0]
        raw_node_2 = node_2.split('_BUS_')[0]
        try:
            distance = self.saved_distances[(raw_node_1, raw_node_2)]
        except:
            distance = self.distance_measure(raw_node_1, raw_node_2)
            self.saved_distances[(raw_node_1, raw_node_2)] = distance
        return distance
        
        
graph = BusPlan()

### Special search algorithm for testing bus route planner

In [ ]:
from collections import deque
import time

graph = BusPlan() # Make the bus version of the planning graph


def step_cost(parent, child):
    """
    Step cost expressed as time
    """
    total_cost = 0
    bus_speed = 30
    walking_speed = 5
    waiting_time = 15 #Approximate time waiting for a bus
    parent_bus = False
    child_bus = False
    
    distance = graph.bus_distance(parent, child)
    
    #Determine if parent and child are bus nodes
    try:
        parent_bus = parent.split('_BUS_')[1]
        parent_bus = True
    except:
        pass
    try:
        child_bus = child.split('_BUS_')[1]
        child_bus = True
    except:
        pass
    
    # Bus travel step cost
    if parent_bus == True and child_bus == True:
        if parent.split('_BUS_')[1] == child.split('_BUS_')[1]:
            return (distance / bus_speed)
    
    #Embarking bus cost
    if parent_bus == False and child_bus == True:
        return waiting_time
    
    # Walking cost
    return (distance / walking_speed)

def heuristic_cost(goal, child):
    return graph.bus_distance(goal, child) / 60

def bus_cost_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    print("Start Search...")
    while not frontier.empty():
        current = frontier.get()
        #Check if the goal has been expanded
        if goal == current:
            print("Solution Found")
            break
        for child in graph.bus_neighbours(current):
            child_cost = cost_so_far[current] + step_cost(current, child)
            if child not in came_from or child_cost < cost_so_far[child]:
                cost_so_far[child] = child_cost
                priority = child_cost + heuristic_cost(goal, child)
                frontier.put(child, priority)  
                came_from[child] = current
        if frontier.empty():
            print("No Solution")    
    graph.save_distances() # Save any calculations for future use.
    return came_from

def walking_cost_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    print("Start Search...")
    while not frontier.empty():
        current = frontier.get()
        #Check if the goal has been expanded
        if goal == current:
            print("Solution Found")
            break
        for child in graph.node_neighbours(current):
            child_cost = cost_so_far[current] + graph.distance_measure(current, child)
            if child not in came_from or child_cost < cost_so_far[child]:
                cost_so_far[child] = child_cost
                priority = child_cost + graph.distance_measure(current, child)
                frontier.put(child, priority)  
                came_from[child] = current
        if frontier.empty():
            print("No Solution")    
    graph.save_distances() # Save any calculations for future use.
    return came_from
            
def path_construction(start, node, search_results):
    if node == start:
        return [node]
    else:
        list_ = [node]
        list_.extend(path_construction(start, search_results[node], search_results))
        return list_
    
def reconstruct_path(came_from, start, goal):
    current = goal
    path = [current]
    while current != start:
        current = came_from[current]
        path.append(current)
    path.append(start) # optional
    path.reverse() # optional
    return path

def bus_node(node):
    try:
        node.split('_BUS_')[1]
        return True
    except:
        return False

def summary_statistics(path):
    distance = 0
    bus_changes = 0
    bus_stops = 0
    walking_distnace = 0
    parent_node = path[0]
    straight_line = graph.bus_distance(parent_node, path[-1])
    for node in path[1:]:
        distance = distance + graph.bus_distance(parent_node, node) # Disntance of route
        if bus_node(parent_node) == False and bus_node(node) == True: # Number of bus changes
            bus_changes = + bus_changes + 1
        if bus_node(node) == True:
            if graph.bus_stop(node.split('_BUS_')[0]) == True:
                bus_stops = bus_stops + 1
        parent_node = node
    return distance, straight_line, bus_changes, bus_stops

#Define Problem
#start =  '3244944628'
#goal = '2368845458'
start =  '1374738772'
goal = '1056171178'

start_time = time.time()
result = bus_cost_search(graph, start, goal)
#result = walking_cost_search(graph, start, goal)
print("--- %s seconds ---" % (time.time() - start_time))
path_1 = path_construction(start, goal, result)
distance, straight_line, bus_changes, bus_stops = summary_statistics(path_1)

In [ ]:
path_1

In [ ]:
# Seconds with empty dictionary 808
# Seconds with full dictionary 575
# Modified with full dictionary 571
# Seconds with a* and empty dictionary 13
# Seconds with a* and full dictionary 9
# Seconds a* with no dictionary 79
# Seconds a* with new save function 42

In [ ]:
import folium
m = folium.Map(location=[ 59.9139, 10.7522], zoom_start=14)
folium.GeoJson(graph.node_path_geojson(path_1),
    style_function=lambda x: {
        'color' : 'black',
        'weight' : 3,
        'opacity': 1,
        'fillColor' : 'black'
        }).add_to(m)
m

## Search Algorithms

In [ ]:
from time import time
from collections import deque

def breadth_first_search(graph, start, goal):
    frontier = deque([])
    frontier.append(start)
    came_from = {}
    came_from[start] = None
    
    while len(frontier) > 0:
        current = frontier.popleft()
        if current == goal:
            print("Solution Found")
            break
        children = graph.node_neighbours(current)
        for child in children:
            if child not in came_from:
                frontier.append(child)
                came_from[child] = current
    return came_from

def uniform_cost_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        #Check if the goal has been expanded
        if goal == current:
            print("Solution Found")
            break
        
        for child in graph.node_neighbours(current):
            child_cost = cost_so_far[current] + graph.distance_measure(current, child)
            if child not in came_from or child_cost < cost_so_far[child]:
                priority = child_cost
                frontier.put(child, priority)
                came_from[child] = current
                cost_so_far[child] = child_cost
        if frontier.empty():
            print("No Solution")
                
    return came_from

def a_star_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        #Check if the goal has been expanded
        if goal == current:
            print("Solution Found")
            break
        
        for child in graph.node_neighbours(current):
            child_cost = cost_so_far[current] + graph.distance_measure(current, child)
            if child not in came_from or child_cost < cost_so_far[child]:
                priority = child_cost
                frontier.put(child, priority)
                came_from[child] = current
                cost_so_far[child] = child_cost + graph.distance_measure(goal, child)
        if frontier.empty():
            print("No Solution")
                
    return came_from

def path_construction(start, node, search_results):
    if node == start:
        return [node]
    else:
        list_ = [node]
        list_.extend(path_construction(start, search_results[node], search_results))
        return list_

In [ ]:
#Define Problem
start =  80005
goal = 260365663
search_results = breadth_first_search(graph, start, goal)
path_1 = path_construction(start, goal, search_results)
#search_results = uniform_cost_search(graph, start, goal)
#path_2 = path_construction(start, goal, search_results)
#search_results = a_star_search(graph, start, goal)
#path_2 = path_construction(start, goal, search_results)

In [ ]:
import psycopg2
from psycopg2.extras import RealDictCursor
import json
conn_string = "host='localhost' dbname='osm' user='dcromp'"
# print the connection string we will use to connect
print ("Connecting to database\n->%s" % (conn_string))
 
# get a connection, if a connect cannot be made an exception will be raised here
conn = psycopg2.connect(conn_string)
 
# conn.cursor will return a cursor object, you can use this cursor to perform queries
cursor = conn.cursor()
print ("Connected!\n")

In [ ]:
# execute our Query
cursor.execute("select relation_id from relation_members where member_id = 266052566")

In [ ]:
records = cursor.fetchall()

In [ ]:
records

In [ ]:
records[0][0].replace('POINT(', ' ').replace(')', '').split(' ')[1:3]

In [ ]:
getattr(Point, 'POINT(10.8501136 59.9435552)')

In [ ]:
test = tagger(records[0][5])
print(test)

In [ ]:
def tagger(self, string):
        """
        Converts and osmosis tag into a python dictionary

        Args:
            Osmosis tag, Example: '"source"=>"bing", "highway"=>"crossing"'
        Returns:
            Python dictionary
            {'source':'bing', 'highway':'crossing'}
        """
        tag_list = string.replace('"','' ).replace(' ','' ).split(',')
        tuples = (x.split('=>') for x in tag_list) # Each tag value pair is now a list of tuples
        return{tuple_[0]:tuple_[1] for tuple_ in tuples}

In [ ]:
graph.node_neighbours(80005)

In [ ]:
from osmapi import OsmApi
from geojson import LineString, GeometryCollection, Point
import folium
from collections import deque
import bz2

In [ ]:
MyApi = OsmApi()

def Map(	self, min_lon, min_lat, max_lon, max_lat)
Download data in bounding box. Returns list of dict { type: node|way|relation, data: {} }.

In [ ]:
highway_tags = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential',
                            'service', 'motorway_link', 'trunk_link', 'primary_link', 'secondary_link', 'tertiary_link',
                            'living_street', 'pedestrian', 'road']
#Raw Data
map_data = MyApi.Map(22.216864,60.416923,22.286005,60.451442)
way_data = [row for row in map_data if row['type'] == 'way']
node_data = [row for row in map_data if row['type'] == 'node']
highway_raw = [row for row in way_data if 'highway' in row['data']['tag'].keys()]
highway_data = [row for row in highway_raw for tag in highway_tags if tag in row['data']['tag']['highway']]
link_data = [row for row in way_data if 'highway' in row['data']['tag'].keys() and 'link' in row['data']['tag']['highway']]

In [ ]:
#Helper functions
def node2loc(node_id):
    for node in node_data:
        if node_id == node['data']['id']:
            return (node['data']['lon'], node['data']['lat'])

In [ ]:
#All Roads to GeoJson
roads = []
for highway in highway_data:
    road = []
    nodes = highway['data']['nd'] #List of nodes in highway
    for node in nodes:
        road.append(node2loc(node))
    roads.append(LineString(road))
geo_json = GeometryCollection(roads)

In [ ]:
m = folium.Map(location=[ 60.4518, 22.2666], zoom_start=15)
folium.GeoJson(geo_json,
    style_function=lambda x: {
        'color' : 'black',
        'weight' : 3
        ,
        'opacity': 1,
        'fillColor' : 'black',
        }).add_to(m)

In [ ]:
m

In [ ]:
#Road turn
turns = set()
for highway_1 in highway_data:
    nodes_1 = highway_1['data']['nd']
    for node_1 in nodes_1:
        for highway_2 in highway_data: 
            if highway_2 != highway_1:
                nodes_2 = highway_2['data']['nd']
                for node_2 in nodes_2:
                    if node_2 == node_1:
                        turns.add(node2loc(node_1))

turn_geojson = GeometryCollection([Point(turn) for turn in list(turns)])

In [ ]:
m = folium.Map(location=[60.4518, 22.2666], zoom_start=15)
folium.GeoJson(geo_json,
    style_function=lambda x: {
        'color' : 'black',
        'weight' : 3
        ,
        'opacity': 1,
        'fillColor' : 'black',
        }).add_to(m)
folium.GeoJson(turn_geojson,
    style_function=lambda x: {
        'color' : 'black',
        'weight' : 3
        ,
        'opacity': 1,
        'fillColor' : 'black',
        }).add_to(m)

In [ ]:
m

In [ ]:
from osmapi import OsmApi
from geojson import LineString, GeometryCollection, Point
from math import radians, sin, cos, sqrt, asin
import heapq
import numpy as np

class GraphPlan(object):
    """ Process data in a bounding box to produce data structures needed for
        a graph search
    """
    def __init__(self, lon1,  lat1, lon2, lat2):
        self.MyApi = OsmApi()
        
        #List of allowed road types
        self.highway_tags = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential',
                            'service', 'motorway_link', 'trunk_link', 'primary_link', 'secondary_link', 'tertiary_link',
                            'living_street', 'pedestrian', 'road']
        
        #Extract basic data units
        self.bounding_box = [lon1,  lat1, lon2, lat2]
        self.map_data = self.MyApi.Map(lon1,  lat1, lon2, lat2)
        self.way_data = [row for row in self.map_data if row['type'] == 'way']
        self.node_data = [row for row in self.map_data if row['type'] == 'node']
        self.highway_data = [row for row in self.way_data if 'highway' in row['data']['tag'].keys()]
        #self.highway_data = [row for row in self.highway_raw for tag in self.highway_tags if tag in row['data']['tag']['highway']]
        
        #Node_id to lon lat
        self.node_ids = [node['data']['id'] for node in self.node_data]
        #self.node_id_loc = {node_id:self.node2loc(node_id) for node_id in self.node_ids}
        
        #Dictionary of road_id and list of node ID's
        self.roads = {highway['data']['id']:highway['data']['nd'] for highway in self.highway_data}
        
        #Dictionary of node_id and list of road_ID's
        self.node_roads = self.__node_roads_dict()
        
        #List of unique nodes at road changes or turns
        self.turns = self.__turn_extractor(self.roads)
        
        #Road_id and their turns
        self.road_turns = self.__road_turns(self.roads)
        
        self.neighbours = self.__neigbour_nodes()
        
        
    #Private Methods
    def __road_turns(self, roads):
        road_turns = {}
        for key in self.roads.keys():
            temp_turns = []
            for node in roads[key]:
                if node in self.turns:
                    temp_turns.append(node)
            road_turns[key] = temp_turns
        return road_turns
    
    def __turn_extractor(self, roads):
        turns = set()
        for road_1 in self.roads.keys():
            nodes_1 = self.roads[road_1]
            for node_1 in nodes_1:
                for road_2 in self.roads.keys(): 
                    if road_2 != road_1:
                        nodes_2 = self.roads[road_2]
                        for node_2 in nodes_2:
                            if node_2 == node_1:
                                turns.add(node_1)
        return list(turns)
    
    def __neigbour_nodes(self):
        neighbours_dict = {}
        for road in self.roads:
            for index, node in enumerate(self.roads[road]):
                forward_node = index + 1
                backward_node = index - 1
                try:
                    forward = self.roads[road][forward_node]
                    if node not in neighbours_dict:
                        neighbours_dict[node] = [forward]
                    else:
                        list_ = neighbours_dict[node]
                        list_.append(forward)
                        neighbours_dict[node] = list_
                except:
                    pass
                if backward_node >= 0:
                    try:
                        backward = self.roads[road][backward_node]
                        if node not in neighbours_dict:
                            neighbours_dict[node] = [backward]
                        else:
                            list_ = neighbours_dict[node]
                            list_.append(backward)
                            neighbours_dict[node] = list_
                    except:
                        pass 
        return neighbours_dict
    
    def __node_roads_dict(self):
        node_roads = {}
        for road in list(self.roads.keys()):
            for node in self.roads[road]:
                if node not in list(node_roads.keys()):
                    node_roads[node] = [road]
                else:
                    list_ = node_roads[node]
                    list_.append(road)
                    node_roads[node] = list_
        return node_roads
    
    def __haversine(self, lat1, lon1, lat2, lon2):
        R = 6372.8 # Earth radius in kilometers
        dLat = radians(lat2 - lat1)
        dLon = radians(lon2 - lon1)
        lat1 = radians(lat1)
        lat2 = radians(lat2)
        a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
        c = 2*asin(sqrt(a))
        return R * c
    
    #Helper Methods
    def node2loc(self, node_id):
        for node in self.node_data:
            if node_id == node['data']['id']:
                return (node['data']['lon'], node['data']['lat'])
    
    def node2road(self, node_id):
        return self.node_roads[node_id] #List of roads node is in
    
    def neighbour_list(self, node_id):
        return self.neighbours[node_id]
    
    def node_path_geojson(self, node_list):
        roads = []
        for node in node_list:
            roads.append(self.node2loc(node))
        return LineString(roads)
    
    def distance_measure(self, node1, node2):
        lon1, lat1 = self.node2loc(node1)
        lon2, lat2 = self.node2loc(node2)
        lon = lon1 - lon2
        lat = lat1 - lat2
        return np.sqrt(np.square(lon) + np.square(lat))
    
    def node_distance(self, node1, node2):
        lon1, lat1 = self.node2loc(node1)
        lon2, lat2 = self.node2loc(node2)
        distance = self.__haversine(lat1, lon1, lat2, lon2)
        return distance
    
class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]
    
class BusGraph(GraphPlan):
    """
    Turns bus routes into nodes for the graph search
    """
    def __init__(self, lon1,  lat1, lon2, lat2):
        GraphPlan.__init__(self, lon1,  lat1, lon2, lat2 )
        
        #Extract all raw bus routes
        self.route_data = [row for row in self.map_data if row['type'] == 'route']
        #self.route_data = [row for row in self.way_data if 'route' in row['data']['tag'].keys()]
        #self.bus_data = [row for row in self.route_data if 'bus' in row['data']['tag']['route']]

In [ ]:
#Load data
#graph = GraphPlan(22.216864,60.416923,22.286005,60.451442)

graph = BusGraph(0.738144,51.805537,0.754623,51.822623)

In [ ]:
graph.map_data

In [ ]:
#Define Problem
start =  249745223
goal = 4756728248

In [ ]:
#Define Problem
start =  249745223
goal = 4756728248
search_results = breadth_first_search(graph, start, goal)
path_1 = path_construction(start, goal, search_results)
#search_results = uniform_cost_search(graph, start, goal)
#path_2 = path_construction(start, goal, search_results)
#search_results = a_star_search(graph, start, goal)
#path_2 = path_construction(start, goal, search_results)

In [ ]:
graph.neighbour_list(1101337726)

In [ ]:
graph.neighbour_list(21646150)

In [ ]:
graph.roads[42684571]

In [ ]:
graph.node2road(21646230)

In [ ]:
graph.roads[22649715]

In [ ]:
path_1

In [ ]:
graph.node_path_geojson([21855869,
 554190049,
 1252511515,
 30070345,
 1252511439,
 1274616355,
 1252511546,
 1252511468,
 21402480,
 256371984,
 597376717,
 1334702233,
 21402481,
 984624826,
 1334702231,
 283313787,
 558546949,
 215546743,
 215546699,
 1334702241,
 471079552,
 21855854,
 530866234,
 756885115,
 1252511533,
 21855855,
 1101337653,
 21646201,
 3223179595,
 2601642846,
 3223179632,
 3223179631,
 597376713,
 1101337825,
 1099808365,
 1099808361,
 1101337543,
 1101337632,
 1252511543,
 21646150])

In [ ]:
graph.roads

In [ ]:
graph.node_path_geojson([21646150, 1101337726, 243674925, 533774011, 32919954, 30390838, 21646152])

In [ ]:
neighbour_nodes = set() #Only collect unique neighbours
way_list = test.node2way(338507468)
for way_id in way_list:
    way = test.ways[way_id]
    way_index = way.index(338507468)
    forward_node = way_index + 1
    backward_node = way_index - 1
    try:
        forward = way[forward_node]
        neighbour_nodes.add(forward)
    except:
        pass
    try:
        if backward_node > 0:
            backward = way[backward_node]
            neighbour_nodes.add(backward)
    except:
        pass

In [ ]:
neighbour_nodes

In [ ]:
temp = [1,2,3]
temp[3]

In [ ]:
test.ways

In [ ]:
m

In [ ]:
deque([2,4]).sort()

In [ ]:
graph = Aplanner(0.738144,51.809888,0.750504,51.822623)

#Define Problem
start = 308546906
goal = 316303594

search_path = []
explored_nodes = set()


def a_star_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
            
        current = frontier.get()
        
        #Check if the goal has been expanded
        if goal == came_from[current]:
            print("Solution Found")
            break
        
        children = graph.neigbour_nodes(current)
        
        for child in children:
            
            child_cost = cost_so_far[current] + graph.node_distance(current, child) - graph.node_distance(child, goal)
            
            if child not in came_from or child_cost < cost_so_far[child]:
                
                priority = child_cost
                frontier.put(child, priority)
                came_from[child] = current
                cost_so_far[child] = child_cost

                
    return came_from

search_results = a_star_search(graph, start, goal)
path = path_construction(start, goal, search_results)

In [ ]:
for node in test.turns:
    print(test.neigbour_nodes(node))

In [ ]:
test.ways[25643156]

In [ ]:
node_ways = {}
node_list = []

for way in list(test.ways.keys()):
    for node in test.ways[way]:
        if node not in list(node_ways.keys()):
            node_ways[node] = [way]
            node_list.append(node)
        else:
            print("Node already in dict")
            list_ = node_ways[node]
            list_.append(way)
            node_ways[node] = list_
#node_ways

In [ ]:
test.ways[32702976]

In [ ]:
print(len(set(node_list)))
print(len(node_list))

In [ ]:
test.turns

In [ ]:
test.turns

In [ ]:
if 1583202304 in test.ways.keys():
    print('yes')

In [ ]:
for key in node_ways.keys():
    if len(node_ways[key]) > 1:
        print (len(node_ways[key]))

In [ ]:
test.node_ways[1583202304]

In [ ]:
def node2loc(node_id):
    for node in node_data:
        if node_id == node['data']['id']:
            return (node['data']['lon'], node['data']['lat'])

In [ ]:
node_data[0]

In [ ]:
for dict_ in map_nodes:
    if dict_['type'] == 'way':
        way = dict_
        if 'highway' in way['data']['tag'].keys():
            print(way, '\n')

In [ ]:
set_ = set()
for dict_ in map_nodes:
    set_.add(tuple(dict_['type']))
    